In [1]:
import numpy as np
import arviz as az
from line_profiler import LineProfiler
from arviz.data import load_arviz_data,from_dict,convert_to_inference_data
from arviz.stats.stats import r2_score, hpd, _gpdfit, _gpinv,_ic_matrix, waic,psislw,loo,compare
from arviz.stats.diagnostics import ess
from arviz.stats.stats_utils import logsumexp as _logsumexp
import scipy.stats as st
import numba
import math
from numpy import sin, cos
import pandas as pd
from scipy.optimize import minimize
import warnings
from pytest import *

ModuleNotFoundError: No module named 'atomicwrites'

In [5]:
data_1 = np.random.randn(10_000,1000)
data_2 = np.random.randn(1_000_000)
school = load_arviz_data("centered_eight").posterior["mu"].values

In [3]:
'''********************************************hpd********************************************************************'''

'********************************************hpd********************************************************************'

In [6]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(data_1, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 2.60059 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: hpd at line 249

Line #      Hits         Time  Per Hit   % Time  Line Contents
   249                                           def hpd(ary, credible_interval=0.94, circular=False):
   250                                               """
   251                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   252                                           
   253                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   254                                               for unimodal distributions.
   255                                           
   256                                               Parameters
   257                                               ----------
   258                                               x : Numpy a

In [7]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(data_2, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.594237 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: hpd at line 249

Line #      Hits         Time  Per Hit   % Time  Line Contents
   249                                           def hpd(ary, credible_interval=0.94, circular=False):
   250                                               """
   251                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   252                                           
   253                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   254                                               for unimodal distributions.
   255                                           
   256                                               Parameters
   257                                               ----------
   258                                               x : Numpy 

In [6]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(school, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.086086 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: hpd at line 249

Line #      Hits         Time  Per Hit   % Time  Line Contents
   249                                           def hpd(ary, credible_interval=0.94, circular=False):
   250                                               """
   251                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   252                                           
   253                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   254                                               for unimodal distributions.
   255                                           
   256                                               Parameters
   257                                               ----------
   258                                               x : Numpy 

In [7]:
'''
Bottlenecks:
    1)scipy.stats.circmean
    2)numpy arctan2
'''

'\nBottlenecks:\n    1)scipy.stats.circmean\n    2)numpy arctan2\n'

In [6]:
def _circ_mean(ary, high, low):
    ary = np.asarray(ary)
    if ary.size==0:
        return np.nan, np.nan
    pi = np.pi
    angles = (ary-low)*2*pi/(high-low)
    S = sinusoidal(angles)
    C = cosine(angles)
    res = np.arctan2(S,C)
    mask = res < 0
    if mask.ndim > 0:
        res[mask] += 2*pi
    elif mask:
        res += 2*pi
    return res*(high - low)/2.0/pi + low

@numba.jit(nopython=True)
def sinusoidal(x):
    summ = 0
    x = x.flatten()
    for i in range(0,len(x)):
        summ = summ+math.sin(x[i])
    return summ
@numba.jit(nopython=True)
def cosine(x):
    summ = 0
    x = x.flatten()
    for i in range(0,len(x)):
        summ = summ+math.cos(x[i])
    return summ


In [12]:
%timeit sinusoidal(data_1)

379 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit np.cumsum(np.sin(data_1))

150 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit sinusoidal(data_2)

33.4 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit np.sin(data_2)

2.81 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%timeit cosine(data_1)

408 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit np.cos(data_1)

61.2 ms ± 6.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit cosine(data_2)

32.6 ms ± 379 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit np.cos(data_2)

2.84 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
np.allclose(_circ_mean(data_1, np.pi, -np.pi), st.circmean(data_1, np.pi, -np.pi))

True

In [18]:
np.allclose(_circ_mean(data_2, np.pi, -np.pi), st.circmean(data_2, np.pi, -np.pi))

True

In [19]:
np.allclose(_circ_mean(school, np.pi, -np.pi), st.circmean(school, np.pi, -np.pi))

True

In [20]:
%timeit _circ_mean(data_1, np.pi, -np.pi)

1.07 s ± 60.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit st.circmean(data_1, np.pi, -np.pi)

250 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit _circ_mean(data_2, np.pi, -np.pi)

97.3 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit st.circmean(data_2, np.pi, -np.pi)

20.3 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit _circ_mean(school, np.pi, -np.pi)

189 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%timeit st.circmean(school, np.pi, -np.pi)

219 µs ± 22.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
'''Performance of _circ_mean is a worse on larger datasets.
   On schools, the performance is much better.
   ¯\_(ツ)_/¯
   Let's see the overall perfomance of hpd
'''

"Performance of _circ_mean is a worse on larger datasets.\n   On schools, the performance is much better.\n   ¯\\_(ツ)_/¯\n   Let's see the overall perfomance of hpd\n"

In [15]:
def hpd_new(ary, credible_interval=0.94, circular=False):
    if ary.ndim > 1:
        hpd_array = np.array(
            [hpd(row, credible_interval=credible_interval, circular=circular) for row in ary.T]
        )
        return hpd_array
    # Make a copy of trace
    ary = ary.copy()
    n = len(ary)

    if circular:
        mean = _circ_mean(ary, high=np.pi, low=-np.pi)
        ary = ary - mean
        ary = np.arctan2(np.sin(ary), np.cos(ary))

    ary = np.sort(ary)
    interval_idx_inc = int(np.floor(credible_interval * n))
    n_intervals = n - interval_idx_inc
    interval_width = ary[interval_idx_inc:] - ary[:n_intervals]

    if len(interval_width) == 0:
        raise ValueError(
            "Too few elements for interval calculation. "
            "Check that credible_interval meets condition 0 =< credible_interval < 1"
        )

    min_idx = np.argmin(interval_width)
    hdi_min = ary[min_idx]
    hdi_max = ary[min_idx + interval_idx_inc]

    if circular:
        hdi_min = hdi_min + mean
        hdi_max = hdi_max + mean
        hdi_min = np.arctan2(np.sin(hdi_min), np.cos(hdi_min))
        hdi_max = np.arctan2(np.sin(hdi_max), np.cos(hdi_max))

    return np.array([hdi_min, hdi_max])

In [28]:
np.allclose(hpd_new(school, 0.95,True), az.stats.hpd(school, 0.95,True))

True

In [29]:
np.allclose(hpd_new(data_1, 0.95,True), az.stats.hpd(data_1, 0.95,True))

True

In [30]:
np.allclose(hpd_new(data_2, 0.95,True), az.stats.hpd(data_2, 0.95,True))

True

In [31]:
%timeit hpd_new(school, 0.95,True)

43.7 ms ± 2.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit az.stats.hpd(school, 0.95,True)

42.8 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit hpd_new(data_1, 0.95,True)

2.37 s ± 127 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit az.stats.hpd(data_1, 0.95,True)

2.25 s ± 145 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit hpd_new(data_2, 0.95,True)

375 ms ± 25.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%timeit az.stats.hpd(data_2, 0.95,True)

304 ms ± 8.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
'''As seen above, the performance of numba is better with school dataset.
The performance with large datasets is a bit unstable.'''

'As seen above, the performance of numba is better with school dataset.\nThe performance with large datasets is a bit unstable.'

In [38]:
'''******************************************R2_SCORE***************************************************************'''

'******************************************R2_SCORE***************************************************************'

In [39]:
data_1 = np.random.randn(1000,1000)
data_2 = np.random.randn(1_000_000)
data_3 = np.random.randn(1000,1000)
data_4 = np.random.randn(1_000_000)
school = load_arviz_data("centered_eight").posterior["mu"].values
n_school = load_arviz_data("non_centered_eight").posterior["mu"].values

In [40]:
lp = LineProfiler()
wrapper = lp(r2_score)
wrapper(data_1, data_3)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.130522 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: r2_score at line 565

Line #      Hits         Time  Per Hit   % Time  Line Contents
   565                                           def r2_score(y_true, y_pred):
   566                                               """R² for Bayesian regression models. Only valid for linear models.
   567                                           
   568                                               Parameters
   569                                               ----------
   570                                               y_true: : array-like of shape = (n_samples) or (n_samples, n_outputs)
   571                                                   Ground truth (correct) target values.
   572                                               y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
   573                                                   Estimated target values.
   574 

In [41]:
lp = LineProfiler()
wrapper = lp(r2_score)
wrapper(data_2, data_4)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.032282 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: r2_score at line 565

Line #      Hits         Time  Per Hit   % Time  Line Contents
   565                                           def r2_score(y_true, y_pred):
   566                                               """R² for Bayesian regression models. Only valid for linear models.
   567                                           
   568                                               Parameters
   569                                               ----------
   570                                               y_true: : array-like of shape = (n_samples) or (n_samples, n_outputs)
   571                                                   Ground truth (correct) target values.
   572                                               y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
   573                                                   Estimated target values.
   574 

In [42]:
@numba.njit
def _var_1d(data):
    a,b = 0,0
    for i in data:
        a = a+i
        b = b+i*i
    return b/len(data)-((a/len(data))**2)

@numba.jit
def _var_2d(data):
    a,b = data.shape
    var = np.zeros(b)
    for i in range(0,b):
        var[i] = _var_1d(data[:,i])
    return var

In [43]:
_var_1d(data_2)

1.0039432724541626

In [44]:
np.var(data_2)

1.0039432724541426

In [45]:
np.allclose(_var_1d(data_2), np.var(data_2))

True

In [46]:
np.allclose(_var_1d(data_4), np.var(data_4))

True

In [47]:
%timeit _var_1d(data_2-data_4)

6.51 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%timeit np.var(data_2-data_4)

12.5 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
np.allclose(_var_1d(data_2-data_4),np.var(data_2-data_4))

True

In [50]:
%timeit _var_2d(school)

51.4 µs ± 5.21 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit np.var(school,0)

58 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [52]:
%timeit _var_2d(data_1)

4.6 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%timeit np.var(data_1,0)

8.57 ms ± 661 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%timeit _var_2d(data_3-data_1)

9.47 ms ± 417 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%timeit np.var(data_3-data_1,0)

22.2 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
np.allclose(_var_2d(school), np.var(school,0))

True

In [57]:
np.allclose(_var_2d(data_1), np.var(data_1,0))

True

In [58]:
np.allclose(_var_2d(data_3), np.var(data_3,0))

True

In [59]:
np.allclose(_var_2d(data_3-data_1), np.var(data_3-data_1,0))

True

In [60]:
np.allclose(_var_2d(data_1-data_3), np.var(data_1-data_3,0))

True

In [61]:
'''Numba is doing wonders with variance. Lets inspect mean'''

'Numba is doing wonders with variance. Lets inspect mean'

In [62]:
@numba.njit
def mean_1d(data):
    summ = 0
    for i in data:
        summ = summ+i
    return summ/len(data)

In [63]:
mean_1d(data_2-data_4)

-0.0017768132660067894

In [64]:
np.mean(data_2-data_4)

-0.0017768132660068384

In [65]:
%timeit mean_1d(data_2-data_4)


6.05 ms ± 361 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
%timeit np.mean(data_2-data_4)

5.94 ms ± 326 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [67]:
'''Numba is not suitable here. Lets see the overall r2_score performance'''

'Numba is not suitable here. Lets see the overall r2_score performance'

In [68]:
def r2_score_new(y_true, y_pred):
    if y_pred.ndim == 1:
        var_y_est = _var_1d(y_pred)
        var_e = _var_1d(y_true - y_pred)
    else:
        var_y_est = _var_1d(y_pred.mean(0))
        var_e = _var_2d(y_true - y_pred)

    r_squared = var_y_est / (var_y_est + var_e)

    return pd.Series([np.mean(r_squared), np.std(r_squared)], index=["r2", "r2_std"])

In [69]:
%timeit r2_score_new(data_2, data_4)

7.39 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [70]:
%timeit az.stats.r2_score(data_2, data_4)

21 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
%timeit r2_score_new(data_1, data_3)

9.21 ms ± 886 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
%timeit az.stats.r2_score(data_1, data_3)

18.4 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [73]:
%timeit r2_score_new(school, n_school)

597 µs ± 6.01 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [74]:
%timeit az.stats.r2_score(school, n_school)

712 µs ± 56.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
df_1 = r2_score_new(school, n_school)
df_2 = az.stats.r2_score(school, n_school)

In [76]:
df_1

r2        0.219553
r2_std    0.168670
dtype: float64

In [77]:
df_2

r2        0.219553
r2_std    0.168670
dtype: float64

In [78]:
df_1==df_2

r2        False
r2_std    False
dtype: bool

In [79]:
r2_score_new(data_1, data_3)

r2        0.000501
r2_std    0.000023
dtype: float64

In [80]:
az.stats.r2_score(data_1, data_3)

r2        0.000501
r2_std    0.000023
dtype: float64

In [81]:
'''I have not included the pandas test yet...will include them soon'''

'I have not included the pandas test yet...will include them soon'

In [82]:
'''It can be clearly seen that numba greatly improves the performance of r2_score.
TBH, this was the function which I thought had the least scope of improvement
Thank you mentors.'''

'It can be clearly seen that numba greatly improves the performance of r2_score.\nTBH, this was the function which I thought had the least scope of improvement\nThank you mentors.'

In [83]:
"""""""""""""""""""""""""""""""""""""""""""_gpdfit"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


'_gpdfit'

In [84]:
data = np.abs(np.sort(np.random.randn(100000)))
school = np.abs((np.sort((load_arviz_data("centered_eight").posterior["mu"].values)[1,:])))

In [85]:
lp = LineProfiler()
wrapper = lp(_gpdfit)
wrapper(data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 1.26034 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpdfit at line 488

Line #      Hits         Time  Per Hit   % Time  Line Contents
   488                                           def _gpdfit(ary):
   489                                               """Estimate the parameters for the Generalized Pareto Distribution (GPD).
   490                                           
   491                                               Empirical Bayes estimate for the parameters of the generalized Pareto
   492                                               distribution given the data.
   493                                           
   494                                               Parameters
   495                                               ----------
   496                                               ary : array
   497                                                   sorted 1D data array
   498                                 

In [86]:
lp = LineProfiler()
wrapper = lp(_gpdfit)
wrapper(school)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.003453 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpdfit at line 488

Line #      Hits         Time  Per Hit   % Time  Line Contents
   488                                           def _gpdfit(ary):
   489                                               """Estimate the parameters for the Generalized Pareto Distribution (GPD).
   490                                           
   491                                               Empirical Bayes estimate for the parameters of the generalized Pareto
   492                                               distribution given the data.
   493                                           
   494                                               Parameters
   495                                               ----------
   496                                               ary : array
   497                                                   sorted 1D data array
   498                                

In [87]:
"""Bottleneck at 518"""

'Bottleneck at 518'

In [2]:
@numba.jit
def log_1p(data):
    log_p = np.zeros_like(data)
    for i in range(0,len(data)):
        if data[i]>-1:
            log_p[i] = math.log1p(data[i])
        else:
            log_p[i] = np.nan
    return log_p

In [89]:
np.allclose(log_1p(data),np.log1p(data))

True

In [90]:
np.allclose(log_1p(school), np.log1p(school))

True

In [91]:
%timeit log_1p(data)

1.96 ms ± 87.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [92]:
%timeit np.log1p(data)

245 µs ± 6.61 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [93]:
%timeit log_1p(school)

11.4 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [94]:
%timeit np.log1p(school)

11.1 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [95]:
'''Performance is somewhat unstable, almost half the time it speeds up the process, sometimes its performance is
while in 10% of trials the performance comes down a notch w.r.t to numpy. Performance on large datasets is....slow 
to put it mildly'''

'Performance is somewhat unstable, almost half the time it speeds up the process, sometimes its performance is\nwhile in 10% of trials the performance comes down a notch w.r.t to numpy. Performance on large datasets is....slow \nto put it mildly'

In [96]:
'''overall performance of gpdfit'''

'overall performance of gpdfit'

In [97]:
def _gpdfit_new(ary):
    prior_bs = 3
    prior_k = 10
    n = len(ary)
    m_est = 30 + int(n ** 0.5)

    b_ary = 1 - np.sqrt(m_est / (np.arange(1, m_est + 1, dtype=float) - 0.5))
    b_ary /= prior_bs * ary[int(n / 4 + 0.5) - 1]
    b_ary += 1 / ary[-1]
    
    K = -b_ary[:, None] * ary
    k_ary = np.log1p(K).mean(axis=1) # pylint: disable=no-member Using my own log_1p here throws an exception
    len_scale = n * (np.log(-(b_ary / k_ary)) - k_ary - 1)
    weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)

    # remove negligible weights
    real_idxs = weights >= 10 * np.finfo(float).eps
    if not np.all(real_idxs):
        weights = weights[real_idxs]
        b_ary = b_ary[real_idxs]
    # normalise weights
    weights /= weights.sum()

    # posterior mean for b
    b_post = np.sum(b_ary * weights)
    # estimate for k
    k_post = log_1p(-b_post * ary).mean()  # pylint: disable=invalid-unary-operand-type,no-member
    # add prior for k_post
    k_post = (n * k_post + prior_k * 0.5) / (n + prior_k)
    sigma = -k_post / b_post
    return k_post, sigma

In [98]:
%timeit _gpdfit_new(data)

406 ms ± 3.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%timeit az.stats.stats._gpdfit(data)

402 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
%timeit _gpdfit_new(school)

413 µs ± 7.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [101]:
%timeit az.stats.stats._gpdfit(school)

536 µs ± 54.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [102]:
'''Similar performance on both the datasets. Up for discussion with mentors'''

'Similar performance on both the datasets. Up for discussion with mentors'

In [103]:
"""""""""""""""""""""""""""""""""""""""""""""_gpvinv"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'_gpvinv'

In [6]:
data = np.random.randn(100000)
school = load_arviz_data("centered_eight").posterior["mu"].values[1,:]

In [105]:
lp = LineProfiler()
wrapper = lp(_gpinv)
wrapper(data, 5,2)
lp.print_stats()


Timer unit: 1e-06 s

Total time: 0.058517 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpinv at line 538

Line #      Hits         Time  Per Hit   % Time  Line Contents
   538                                           def _gpinv(probs, kappa, sigma):
   539                                               """Inverse Generalized Pareto distribution function."""
   540                                               # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
   541         1        310.0    310.0      0.5      x = np.full_like(probs, np.nan)
   542         1          2.0      2.0      0.0      if sigma <= 0:
   543                                                   return x
   544         1        969.0    969.0      1.7      ok = (probs > 0) & (probs < 1)
   545         1         74.0     74.0      0.1      if np.all(ok):
   546                                                   if np.abs(kappa) < np.finfo(float).eps:
   547            

In [106]:
lp = LineProfiler()
wrapper = lp(_gpinv)
wrapper(school, 5,2)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.000297 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpinv at line 538

Line #      Hits         Time  Per Hit   % Time  Line Contents
   538                                           def _gpinv(probs, kappa, sigma):
   539                                               """Inverse Generalized Pareto distribution function."""
   540                                               # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
   541         1         43.0     43.0     14.5      x = np.full_like(probs, np.nan)
   542         1          5.0      5.0      1.7      if sigma <= 0:
   543                                                   return x
   544         1         38.0     38.0     12.8      ok = (probs > 0) & (probs < 1)
   545         1         47.0     47.0     15.8      if np.all(ok):
   546                                                   if np.abs(kappa) < np.finfo(float).eps:
   547            

In [107]:
'''Log1p or expm is the bottleneck again'''

'Log1p or expm is the bottleneck again'

In [3]:
@numba.njit
def expm(data):
    expm = np.zeros_like(data)
    for i in range(0,len(data)):
        expm[i] = math.expm1(data[i])
    return expm

In [109]:
np.allclose(expm(data), np.expm1(data))

True

In [110]:
%timeit expm(data)

2.79 ms ± 69.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%timeit np.expm1(data)

246 µs ± 4.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
%timeit expm(school)

13.7 µs ± 322 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [113]:
%timeit np.expm1(school)

13.4 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [114]:
'''numpy is a bit faster
Lets see the overall performance'''

'numpy is a bit faster\nLets see the overall performance'

In [8]:
def _gpinv_new(probs, kappa, sigma):
    """Inverse Generalized Pareto distribution function."""
    # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
    x = np.full_like(probs, np.nan)
    if sigma <= 0:
        return x
    ok = (probs > 0) & (probs < 1)
    if np.all(ok):
        if np.abs(kappa) < np.finfo(float).eps:
            x = -log_1p(-probs)
        else:
            x = expm(-kappa * np.log1p(-probs)) / kappa
        x *= sigma
    else:
        if np.abs(kappa) < np.finfo(float).eps:
            x[ok] = -log_1p(-probs[ok])
        else:
            x[ok] = expm(-kappa * np.log1p(-probs[ok])) / kappa
        x *= sigma
        x[probs == 0] = 0
        if kappa >= 0:
            x[probs == 1] = np.inf
        else:
            x[probs == 1] = -sigma / kappa
    return x

In [9]:
a = _gpinv_new(data,-5,2)

In [10]:
b = az.stats.stats._gpinv(data, -5, 2)

In [11]:
a

array([       nan,        nan,        nan, ..., 0.4       ,        nan,
       0.15525064])

In [12]:
b


array([       nan,        nan,        nan, ..., 0.4       ,        nan,
       0.15525064])

In [13]:
a-b

array([nan, nan, nan, ...,  0., nan,  0.])

In [14]:
%timeit _gpinv_new(data, -5, 2)

3.67 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit az.stats.stats._gpinv(data, -5, 2)

2.85 ms ± 78.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit _gpinv_new(school, -5, 2)

In [124]:
%timeit az.stats.stats._gpinv(school, -5, 2)

51.4 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [125]:
'''numba _gpinv is similar in performance'''

'numba _gpinv is similar in performance'

In [126]:
def generator(dataset_dict,ic='waic'):
    ic_i = "{}_i".format(ic)

    ics = pd.DataFrame()
    names = []
    for name, dataset in dataset_dict.items():
        names.append(name)
        ics = ics.append([waic(dataset, pointwise=True, scale="deviance")])
    ics.index = names
    ics.sort_values(by=ic, inplace=True, ascending=True)
    return ics, ic_i

In [127]:
ics, ic_i = generator({"1":load_arviz_data("centered_eight"),"2":load_arviz_data("non_centered_eight")})

In [128]:
ics

,waic,waic_se,p_waic,warning,waic_i,waic_scale
2,61.302151,2.727291,0.820067,0,"[9.732429123207623, 6.813708378182436, 7.70616...",deviance
1,61.429587,2.689941,0.919548,0,"[9.76127241619229, 6.832600316056821, 7.725554...",deviance


In [129]:
lp = LineProfiler()
wrapper = lp(_ic_matrix)
wrapper(ics,ic_i)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.004775 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _ic_matrix at line 232

Line #      Hits         Time  Per Hit   % Time  Line Contents
   232                                           def _ic_matrix(ics, ic_i):
   233                                               """Store the previously computed pointwise predictive accuracy values (ics) in a 2D matrix."""
   234         1         30.0     30.0      0.6      cols, _ = ics.shape
   235         1        763.0    763.0     16.0      rows = len(ics[ic_i].iloc[0])
   236         1         13.0     13.0      0.3      ic_i_val = np.zeros((rows, cols))
   237                                           
   238         3        203.0     67.7      4.3      for idx, val in enumerate(ics.index):
   239         2       3708.0   1854.0     77.7          ic = ics.loc[val][ic_i]
   240                                           
   241         2         11.0      5.5      0.2          if len(ic

In [130]:
@numba.jit
def loop_lifter(ics,rows,cols):
    ic_i_val = np.zeros((rows, cols))
    for idx, val in enumerate(ics.index):
        ic = ics.loc[val][ic_i]

        if len(ic) != rows:
            raise ValueError("The number of observations should be the same across all models")

        ic_i_val[:, idx] = ic
    return ic_i_val

def _ic_matrix_new(ics, ic_i):
    """Store the previously computed pointwise predictive accuracy values (ics) in a 2D matrix."""
    cols, _ = ics.shape
    rows = len(ics[ic_i].iloc[0])
    return rows, cols, loop_lifter(ics,rows,cols)


In [131]:
%timeit _ic_matrix_new(ics, ic_i)

814 µs ± 116 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
%timeit az.stats.stats._ic_matrix(ics, ic_i)

717 µs ± 72.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [133]:
'''Loop lifitng is not useful in this case. Numba fails at no nopython mode and falls back to pyobject mode.
   Original _ic_matrix is better'''

'Loop lifitng is not useful in this case. Numba fails at no nopython mode and falls back to pyobject mode.\n   Original _ic_matrix is better'

In [134]:
data = np.random.randn(1000,1000,20)
sample_stats = {"log_likelihood":data}
data = from_dict(sample_stats=sample_stats)
school = load_arviz_data("centered_eight")


In [135]:
lp = LineProfiler()
wrapper = lp(waic)
wrapper(data,True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.678223 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: waic at line 865

Line #      Hits         Time  Per Hit   % Time  Line Contents
   865                                           def waic(data, pointwise=False, scale="deviance"):
   866                                               """Calculate the widely available information criterion.
   867                                           
   868                                               Also calculates the WAIC's standard error and the effective number of
   869                                               parameters of the samples in trace from model. Read more theory here - in
   870                                               a paper by some of the leading authorities on model selection
   871                                               dx.doi.org/10.1111/1467-9868.00353
   872                                           
   873                                          

/home/banzee/Desktop/arviz/arviz/stats/stats.py:931: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        
  """


In [136]:
def waic_new(data, pointwise=False, scale="deviance"):
    inference_data = convert_to_inference_data(data)
    for group in ("sample_stats",):
        if not hasattr(inference_data, group):
            raise TypeError(
                "Must be able to extract a {group} group from data!".format(group=group)
            )
    if "log_likelihood" not in inference_data.sample_stats:
        raise TypeError("Data must include log_likelihood in sample_stats")
    log_likelihood = inference_data.sample_stats.log_likelihood

    if scale.lower() == "deviance":
        scale_value = -2
    elif scale.lower() == "log":
        scale_value = 1
    elif scale.lower() == "negative_log":
        scale_value = -1
    else:
        raise TypeError('Valid scale values are "deviance", "log", "negative_log"')

    n_samples = log_likelihood.chain.size * log_likelihood.draw.size
    new_shape = (n_samples, np.product(log_likelihood.shape[2:]))
    log_likelihood = log_likelihood.values.reshape(*new_shape)

    lppd_i = _logsumexp(log_likelihood, axis=0, b_inv=log_likelihood.shape[0])

    vars_lpd = _var_2d(log_likelihood)
    warn_mg = 0
    if np.any(vars_lpd > 0.4):
        warnings.warn(
            """For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        """
        )
        warn_mg = 1

    waic_i = scale_value * (lppd_i - vars_lpd)
    waic_se = (len(waic_i) * _var_1d(waic_i)) ** 0.5
    waic_sum = np.sum(waic_i)
    p_waic = np.sum(vars_lpd)

    if pointwise:
        if np.equal(waic_sum, waic_i).all():  # pylint: disable=no-member
            warnings.warn(
                """The point-wise WAIC is the same with the sum WAIC, please double check
            the Observed RV in your model to make sure it returns element-wise logp.
            """
            )
        return pd.Series(
            data=[waic_sum, waic_se, p_waic, warn_mg, waic_i, scale],
            index=["waic", "waic_se", "p_waic", "warning", "waic_i", "waic_scale"],
        )
    else:
        return pd.Series(
            data=[waic_sum, waic_se, p_waic, warn_mg, scale],
            index=["waic", "waic_se", "p_waic", "warning", "waic_scale"],
        )

In [137]:
waic_new(data,True)

/home/banzee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        


waic                                                    20.0055
waic_se                                               0.0109872
p_waic                                                   20.014
warning                                                       1
waic_i        [1.0001109256408993, 0.9997360431832794, 0.998...
waic_scale                                             deviance
dtype: object

In [138]:
az.stats.waic(data,True)

/home/banzee/Desktop/arviz/arviz/stats/stats.py:931: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        
  """


waic                                                    20.0055
waic_se                                               0.0109872
p_waic                                                   20.014
warning                                                       1
waic_i        [1.0001109256408607, 0.9997360431832267, 0.998...
waic_scale                                             deviance
dtype: object

In [139]:
%timeit  waic_new(data,True)

/home/banzee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        


615 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
%timeit  az.stats.waic(data,True)

/home/banzee/Desktop/arviz/arviz/stats/stats.py:931: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        
  """


749 ms ± 90.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [141]:
%timeit  waic_new(school,True)

2.41 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [142]:
%timeit  az.stats.waic(school,True)

2.57 ms ± 374 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [143]:
"""""""""""""""""""""""""""""""""""""""psislw"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'psislw'

In [144]:
@numba.njit
def max_elem(data):
    max = data[0]
    for i in range(0,len(data)):
        if data[i]>max:
            max = data[i]
    return max

In [145]:
data = np.random.rand(1000,1000)
school = load_arviz_data("centered_eight").posterior["mu"].values

In [146]:
lp = LineProfiler()
wrapper = lp(psislw)
wrapper(data, 0.66)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.789172 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: psislw at line 417

Line #      Hits         Time  Per Hit   % Time  Line Contents
   417                                           def psislw(log_weights, reff=1.0):
   418                                               """
   419                                               Pareto smoothed importance sampling (PSIS).
   420                                           
   421                                               Parameters
   422                                               ----------
   423                                               log_weights : array
   424                                                   Array of size (n_samples, n_observations)
   425                                               reff : float
   426                                                   relative MCMC efficiency, `ess / n`
   427                                           
   428      

In [147]:
'''Lets use our own gpd funcs'''

'Lets use our own gpd funcs'

In [148]:
@numba.jit
def psislw_new(log_weights, reff=1.0):
    rows, cols = log_weights.shape

    log_weights_out = np.copy(log_weights, order="F")
    kss = np.empty(cols)

    # precalculate constants
    cutoff_ind = -int(np.ceil(min(rows / 5.0, 3 * (rows / reff) ** 0.5))) - 1
    cutoffmin = np.log(np.finfo(float).tiny)  # pylint: disable=no-member, assignment-from-no-return
    k_min = 1.0 / 3

    # loop over sets of log weights
    for i, x in enumerate(log_weights_out.T):
        # improve numerical accuracy
        x -= np.max(x)
        # sort the array
        x_sort_ind = np.argsort(x)
        # divide log weights into body and right tail
        xcutoff = max(x[x_sort_ind[cutoff_ind]], cutoffmin)

        expxcutoff = np.exp(xcutoff)
        tailinds, = np.where(x > xcutoff)  # pylint: disable=unbalanced-tuple-unpacking
        x_tail = x[tailinds]
        tail_len = len(x_tail)
        if tail_len <= 4:
            # not enough tail samples for gpdfit
            k = np.inf
        else:
            # order of tail samples
            x_tail_si = np.argsort(x_tail)
            # fit generalized Pareto distribution to the right tail samples
            x_tail = np.exp(x_tail) - expxcutoff
            k, sigma = _gpdfit_new(x_tail[x_tail_si])

            if k >= k_min:
                # no smoothing if short tail or GPD fit failed
                # compute ordered statistic for the fit
                sti = np.arange(0.5, tail_len) / tail_len
                smoothed_tail = _gpinv_new(sti, k, sigma)
                print(smoothed_tail.shape)
                smoothed_tail = np.log(  # pylint: disable=assignment-from-no-return
                    smoothed_tail + expxcutoff
                )
                # place the smoothed tail into the output array
                x[tailinds[x_tail_si]] = smoothed_tail
                # truncate smoothed values to the largest raw weight 0
                x[x > 0] = 0
        # renormalize weights
        x -= _logsumexp(x)
        # store tail index k
        kss[i] = k

    return log_weights_out, kss

In [149]:
psislw_new(data)

(array([[-7.24458336, -6.45542208, -7.22064496, ..., -7.20762862,
         -6.79213841, -6.58060632],
        [-7.41325314, -7.07948432, -7.20591076, ..., -7.16446562,
         -7.068009  , -7.29749703],
        [-6.63120208, -6.92522927, -7.35638611, ..., -7.05805899,
         -7.17555111, -7.27267608],
        ...,
        [-6.62006196, -6.90663181, -7.00081026, ..., -6.63930009,
         -7.04663538, -6.73331676],
        [-7.07982562, -6.46380939, -7.26656309, ..., -6.85599341,
         -7.42139695, -6.85001487],
        [-7.21116532, -6.73833036, -7.37009515, ..., -6.90124615,
         -6.75996199, -6.83192004]]),
 array([-0.73279016, -0.90755503, -0.86620979, -0.64396395, -0.72351762,
        -0.75122911, -0.92520345, -0.64258316, -0.58947724, -0.94671867,
        -0.74093495, -0.76844757, -0.91327719, -0.89761062, -0.99053363,
        -0.69668055, -1.065511  , -0.74380369, -0.80705144, -0.77455233,
        -0.88437539, -0.71495365, -0.90521139, -0.9332444 , -0.8227635 ,
        

In [151]:
%timeit psislw_new(data,0.84)

594 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [152]:
%timeit az.stats.psislw(data,0.84)

577 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [153]:
%timeit psislw_new(school,0.84)

39.2 ms ± 667 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [154]:
%timeit az.stats.psislw(school,0.84)

38.8 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
'''Not much improvement'''

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""LOO"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

In [155]:
data = np.random.randn(1000,1000,20)
mu = np.random.randn(1000,1000)
theta = np.random.randn(1000,1000)
sd = np.random.randn(1000,1000)
posterior = {"mu":mu,"theta":theta,"sd":sd}
sample_stats = {"log_likelihood":data}
data = from_dict(sample_stats=sample_stats,posterior=posterior)
school = load_arviz_data("centered_eight")

In [156]:
data

Inference data with groups:
	> posterior
	> sample_stats

In [157]:
lp = LineProfiler()
wrapper = lp(loo)
wrapper(data,True,0.8)
lp.print_stats()

/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


Timer unit: 1e-06 s

Total time: 5.90912 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: loo at line 309

Line #      Hits         Time  Per Hit   % Time  Line Contents
   309                                           def loo(data, pointwise=False, reff=None, scale="deviance"):
   310                                               """Pareto-smoothed importance sampling leave-one-out cross-validation.
   311                                           
   312                                               Calculates leave-one-out (LOO) cross-validation for out of sample predictive model fit,
   313                                               following Vehtari et al. (2017). Cross-validation is computed using Pareto-smoothed
   314                                               importance sampling (PSIS).
   315                                           
   316                                               Parameters
   317                                               ---

In [158]:
def loo_new(data, pointwise=False, reff=None, scale="deviance"):
    inference_data = convert_to_inference_data(data)
    for group in ("posterior", "sample_stats"):
        if not hasattr(inference_data, group):
            raise TypeError(
                "Must be able to extract a {group} group from data!".format(group=group)
            )
    if "log_likelihood" not in inference_data.sample_stats:
        raise TypeError("Data must include log_likelihood in sample_stats")
    posterior = inference_data.posterior
    log_likelihood = inference_data.sample_stats.log_likelihood
    n_samples = log_likelihood.chain.size * log_likelihood.draw.size
    new_shape = (n_samples, np.product(log_likelihood.shape[2:]))
    log_likelihood = log_likelihood.values.reshape(*new_shape)

    if scale.lower() == "deviance":
        scale_value = -2
    elif scale.lower() == "log":
        scale_value = 1
    elif scale.lower() == "negative_log":
        scale_value = -1
    else:
        raise TypeError('Valid scale values are "deviance", "log", "negative_log"')

    if reff is None:
        n_chains = len(posterior.chain)
        if n_chains == 1:
            reff = 1.0
        else:
            ess_p = ess(posterior, method="mean")
            # this mean is over all data variables
            reff = (
                np.hstack([ess_p[v].values.flatten() for v in ess_p.data_vars]).mean() / n_samples
            )

    log_weights, pareto_shape = psislw_new(-log_likelihood, reff)
    log_weights += log_likelihood

    warn_mg = 0
    if np.any(pareto_shape > 0.7):
        warnings.warn(
            """Estimated shape parameter of Pareto distribution is greater than 0.7 for
        one or more samples. You should consider using a more robust model, this is because
        importance sampling is less likely to work well if the marginal posterior and LOO posterior
        are very different. This is more likely to happen with a non-robust model and highly
        influential observations."""
        )
        warn_mg = 1

    loo_lppd_i = scale_value * _logsumexp(log_weights, axis=0)
    loo_lppd = loo_lppd_i.sum()
    loo_lppd_se = (len(loo_lppd_i) * _var_1d(loo_lppd_i)) ** 0.5

    lppd = np.sum(_logsumexp(log_likelihood, axis=0, b_inv=log_likelihood.shape[0]))
    p_loo = lppd - loo_lppd / scale_value

    if pointwise:
        if np.equal(loo_lppd, loo_lppd_i).all():  # pylint: disable=no-member
            warnings.warn(
                """The point-wise LOO is the same with the sum LOO, please double check
                          the Observed RV in your model to make sure it returns element-wise logp.
                          """
            )
        return pd.Series(
            data=[loo_lppd, loo_lppd_se, p_loo, warn_mg, loo_lppd_i, pareto_shape, scale],
            index=["loo", "loo_se", "p_loo", "warning", "loo_i", "pareto_k", "loo_scale"],
        )

    else:
        return pd.Series(
            data=[loo_lppd, loo_lppd_se, p_loo, warn_mg, scale],
            index=["loo", "loo_se", "p_loo", "warning", "loo_scale"],
        )



In [159]:
%timeit loo_new(data,True)

/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return u

6.08 s ± 96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [160]:
%timeit az.stats.loo(data,True)

/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/banzee/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return u

6.25 s ± 383 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [161]:
%timeit loo_new(school,True)

(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)
(316,)
(326,)
(326,)
(326,)
(326,)

In [162]:
%timeit az.stats.loo(school,True)

23.4 ms ± 449 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [163]:
loo_new(school,True)

(326,)
(326,)
(326,)
(316,)
(326,)


loo                                                    61.4893
loo_se                                                 2.70297
p_loo                                                 0.949409
warning                                                      0
loo_i        [9.77935763951731, 6.840747572952978, 7.726586...
pareto_k     [0.4439131521000332, 0.3284878432072214, 0.529...
loo_scale                                             deviance
dtype: object

In [164]:
 az.stats.loo(school,True)

loo                                                    61.4893
loo_se                                                 2.70297
p_loo                                                 0.949409
warning                                                      0
loo_i        [9.77935763951731, 6.840747572952978, 7.726586...
pareto_k     [0.4439131521000332, 0.3284878432072214, 0.529...
loo_scale                                             deviance
dtype: object

In [165]:
np.finfo(float).eps

2.220446049250313e-16

In [166]:
@numba.njit
def summation(data):
    x = 0
    for i in range(0,len(data)):
        x = x+data[i]
    return x

In [167]:
c = np.random.randn(10000000)

In [168]:
np.allclose(summation(c), np.sum(c))

True

In [169]:
%timeit summation(c)

17.1 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [170]:
%timeit np.sum(c)

9.26 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [171]:
data = np.abs(np.random.randn(100000))

In [172]:
@numba.njit
def logarithm(data):
    log_arr = np.zeros_like(data)
    for i in range(0,len(data)):
        log_arr[i] = math.log(data[i])
    return log_arr

In [173]:
np.allclose(logarithm(data), np.log(data))

True

In [174]:
%timeit logarithm(data)


4.51 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [175]:
%timeit np.log(data)

190 µs ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
'''Not much improvement in performance of loo. Sometimes a bit slower as well'''

In [ ]:
"""""""""""""""""""""""""""""""""""""""Compare"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

In [176]:
school = load_arviz_data("centered_eight")
nschool = load_arviz_data("non_centered_eight")
compare_dict = {"1":school,"2":nschool}

In [177]:
compare(compare_dict, method='stacking')


,waic,p_waic,d_waic,weight,se,dse,warning,waic_scale
2,61.3022,0.820067,0,0.5,2.72729,0,0,deviance
1,61.4296,0.919548,0.127437,0.5,2.68994,0.106882,0,deviance


In [178]:
lp = LineProfiler()
wrapper = lp(compare)
wrapper(compare_dict,'waic','stacking')
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.088127 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: compare at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def compare(
    26                                               dataset_dict,
    27                                               ic="waic",
    28                                               method="BB-pseudo-BMA",
    29                                               b_samples=1000,
    30                                               alpha=1,
    31                                               seed=None,
    32                                               scale="deviance",
    33                                           ):
    34                                               r"""Compare models based on WAIC or LOO cross validation.
    35                                           
    36                                               WAIC is

In [179]:
lp = LineProfiler()
wrapper = lp(compare)
wrapper(compare_dict,'waic','bb-pseudo-bma')
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.246776 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: compare at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def compare(
    26                                               dataset_dict,
    27                                               ic="waic",
    28                                               method="BB-pseudo-BMA",
    29                                               b_samples=1000,
    30                                               alpha=1,
    31                                               seed=None,
    32                                               scale="deviance",
    33                                           ):
    34                                               r"""Compare models based on WAIC or LOO cross validation.
    35                                           
    36                                               WAIC is

In [403]:

def compare_new(
    dataset_dict,
    ic="waic",
    method="BB-pseudo-BMA",
    b_samples=1000,
    alpha=1,
    seed=None,
    scale="deviance",
):
    names = list(dataset_dict.keys())
    scale = scale.lower()
    if scale == "log":
        scale_value = 1
        ascending = False
    else:
        if scale == "negative_log":
            scale_value = -1
        else:
            scale_value = -2
        ascending = True

    if ic == "waic":
        ic_func = waic_new
        df_comp = pd.DataFrame(
            index=names,
            columns=["waic", "p_waic", "d_waic", "weight", "se", "dse", "warning", "waic_scale"],
        )
        scale_col = "waic_scale"

    elif ic == "loo":
        ic_func = loo
        df_comp = pd.DataFrame(
            index=names,
            columns=["loo", "p_loo", "d_loo", "weight", "se", "dse", "warning", "loo_scale"],
        )
        scale_col = "loo_scale"

    else:
        raise NotImplementedError("The information criterion {} is not supported.".format(ic))

    if method.lower() not in ["stacking", "bb-pseudo-bma", "pseudo-bma"]:
        raise ValueError("The method {}, to compute weights, is not supported.".format(method))

    ic_se = "{}_se".format(ic)
    p_ic = "p_{}".format(ic)
    ic_i = "{}_i".format(ic)

    ics = pd.DataFrame()
    names = []
    for name, dataset in dataset_dict.items():
        names.append(name)
        ics = ics.append([ic_func(dataset, pointwise=True, scale=scale)])
    ics.index = names
    ics.sort_values(by=ic, inplace=True, ascending=ascending)

    if method.lower() == "stacking":
        rows, cols, ic_i_val = _ic_matrix(ics, ic_i)
        exp_ic_i = np.exp(ic_i_val / scale_value)
        last_col = cols - 1
        
        def w_fuller(weights):
            np.concatenate((weights, [max(1.0 - np.sum(weights), 0.0)]))

        def log_score(weights):
            w_full = w_fuller(weights)
            score = 0.0
            for i in range(rows):
                k =  np.log(np.dot(exp_ic_i[i], w_full))
                score = score+k
            return -score

        def gradient(weights):
            w_full = w_fuller(weights)
            grad = np.zeros(last_col)
            for k in range(last_col - 1):
                for i in range(rows):
                    N = exp_ic_i[i, k] - exp_ic_i[i, last_col]
                    D = np.dot(exp_ic_i[i], w_full)
                    grad[k] += N/D
            return -grad

        theta = np.full(last_col, 1.0 / cols)
        bounds = [(0.0, 1.0) for _ in range(last_col)]
        constraints = [
            {"type": "ineq", "fun": lambda x: 1.0 - np.sum(x)},
            {"type": "ineq", "fun": np.sum},
        ]

        weights = minimize(
            fun=log_score, x0=theta, jac=gradient, bounds=bounds, constraints=constraints
        )

        weights = w_fuller(weights["x"])
        ses = ics[ic_se]

    elif method.lower() == "bb-pseudo-bma":
        rows, cols, ic_i_val = _ic_matrix(ics, ic_i)
        ic_i_val = ic_i_val * rows

        b_weighting = st.dirichlet.rvs(alpha=[alpha] * rows, size=b_samples, random_state=seed)
        @numba.jit
        def weight_function(ic_i_val,b_weighting,scale_value,b_samples):
            weights = np.zeros((b_samples, cols))
            z_bs = np.zeros_like(weights)
            for i in range(b_samples):
                z_b = np.dot(b_weighting[i], ic_i_val)
                u_weights = np.exp((z_b-np.min(z_b))/scale_value)
                z_bs[i] = z_b  # pylint: disable=unsupported-assignment-operation
                weights[i] = u_weights / np.sum(u_weights)
            return weights,z_bs
            
        weights, z_bs = weight_function(ic_i_val, b_weighting,scale_value, b_samples)
        weights = weights.mean(axis=0)
        ses = pd.Series(z_bs.std(axis=0), index=names)  # pylint: disable=no-member

    elif method.lower() == "pseudo-bma":
        min_ic = ics.iloc[0][ic]
        z_rv = np.exp((ics[ic] - min_ic) / scale_value)
        weights = z_rv / np.sum(z_rv)
        ses = ics[ic_se]

    if np.any(weights):
        min_ic_i_val = ics[ic_i].iloc[0]
        for idx, val in enumerate(ics.index):
            res = ics.loc[val]
            if scale_value < 0:
                diff = res[ic_i] - min_ic_i_val
            else:
                diff = min_ic_i_val - res[ic_i]
            d_ic = np.sum(diff)
            d_std_err = np.sqrt(len(diff) * np.var(diff))
            std_err = ses.loc[val]
            weight = weights[idx]
            df_comp.at[val] = (
                res[ic],
                res[p_ic],
                d_ic,
                weight,
                std_err,
                d_std_err,
                res["warning"],
                res[scale_col],
            )

    return df_comp.sort_values(by=ic, ascending=ascending)

In [ ]:
# Let's see the effect of loop jitting in bb-pseduo-mma

In [404]:
%timeit compare_new(compare_dict,'waic')

435 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [405]:
%timeit az.stats.compare(compare_dict,'waic')

46.4 ms ± 582 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [406]:
#Sheesh, it's 10 times slower.It was expected as nopython failed to compile.
#Let's try to vectorize np.exp((z_b-np.min(z_b))/scale_value)

In [419]:
def compare_new(
    dataset_dict,
    ic="waic",
    method="BB-pseudo-BMA",
    b_samples=1000,
    alpha=1,
    seed=None,
    scale="deviance",
):
    names = list(dataset_dict.keys())
    scale = scale.lower()
    if scale == "log":
        scale_value = 1
        ascending = False
    else:
        if scale == "negative_log":
            scale_value = -1
        else:
            scale_value = -2
        ascending = True

    if ic == "waic":
        ic_func = waic_new
        df_comp = pd.DataFrame(
            index=names,
            columns=["waic", "p_waic", "d_waic", "weight", "se", "dse", "warning", "waic_scale"],
        )
        scale_col = "waic_scale"

    elif ic == "loo":
        ic_func = loo
        df_comp = pd.DataFrame(
            index=names,
            columns=["loo", "p_loo", "d_loo", "weight", "se", "dse", "warning", "loo_scale"],
        )
        scale_col = "loo_scale"

    else:
        raise NotImplementedError("The information criterion {} is not supported.".format(ic))

    if method.lower() not in ["stacking", "bb-pseudo-bma", "pseudo-bma"]:
        raise ValueError("The method {}, to compute weights, is not supported.".format(method))

    ic_se = "{}_se".format(ic)
    p_ic = "p_{}".format(ic)
    ic_i = "{}_i".format(ic)

    ics = pd.DataFrame()
    names = []
    for name, dataset in dataset_dict.items():
        names.append(name)
        ics = ics.append([ic_func(dataset, pointwise=True, scale=scale)])
    ics.index = names
    ics.sort_values(by=ic, inplace=True, ascending=ascending)

    if method.lower() == "stacking":
        rows, cols, ic_i_val = _ic_matrix(ics, ic_i)
        exp_ic_i = np.exp(ic_i_val / scale_value)
        last_col = cols - 1
        
        def w_fuller(weights):
            np.concatenate((weights, [max(1.0 - np.sum(weights), 0.0)]))

        def log_score(weights):
            w_full = w_fuller(weights)
            score = 0.0
            for i in range(rows):
                k =  np.log(np.dot(exp_ic_i[i], w_full))
                score = score+k
            return -score

        def gradient(weights):
            w_full = w_fuller(weights)
            grad = np.zeros(last_col)
            for k in range(last_col - 1):
                for i in range(rows):
                    N = exp_ic_i[i, k] - exp_ic_i[i, last_col]
                    D = np.dot(exp_ic_i[i], w_full)
                    grad[k] += N/D
            return -grad

        theta = np.full(last_col, 1.0 / cols)
        bounds = [(0.0, 1.0) for _ in range(last_col)]
        constraints = [
            {"type": "ineq", "fun": lambda x: 1.0 - np.sum(x)},
            {"type": "ineq", "fun": np.sum},
        ]

        weights = minimize(
            fun=log_score, x0=theta, jac=gradient, bounds=bounds, constraints=constraints
        )

        weights = w_fuller(weights["x"])
        ses = ics[ic_se]

    elif method.lower() == "bb-pseudo-bma":
        rows, cols, ic_i_val = _ic_matrix(ics, ic_i)
        ic_i_val = ic_i_val * rows
        
        @numba.vectorize(nopython=True)
        def z_b_weights(x,min_x,scale_value):
            return np.exp((x-min_x)/scale_value)

        b_weighting = st.dirichlet.rvs(alpha=[alpha] * rows, size=b_samples, random_state=seed)
        weights = np.zeros((b_samples, cols))
        z_bs = np.zeros_like(weights)
        for i in range(b_samples):
            z_b = np.dot(b_weighting[i], ic_i_val)
            u_weights = z_b_weights(z_b, np.min(z_b), scale_value)
            z_bs[i] = z_b  # pylint: disable=unsupported-assignment-operation
            weights[i] = u_weights / np.sum(u_weights)
            
        weights = weights.mean(axis=0)
        ses = pd.Series(z_bs.std(axis=0), index=names)  # pylint: disable=no-member

    elif method.lower() == "pseudo-bma":
        
        @numba.vectorize(nopython=True)
        def z_b_weights(x,min_x,scale_value):
            return np.exp((x-min_x)/scale_value)
        
        min_ic = ics.iloc[0][ic]
        z_rv = z_b_weights(ics[ic].values,min_ic,scale_value)
        
        weights = z_rv / np.sum(z_rv)
        ses = ics[ic_se]

    if np.any(weights):
        min_ic_i_val = ics[ic_i].iloc[0]
        for idx, val in enumerate(ics.index):
            res = ics.loc[val]
            if scale_value < 0:
                diff = res[ic_i] - min_ic_i_val
            else:
                diff = min_ic_i_val - res[ic_i]
            d_ic = np.sum(diff)
            d_std_err = np.sqrt(len(diff) * np.var(diff))
            std_err = ses.loc[val]
            weight = weights[idx]
            df_comp.at[val] = (
                res[ic],
                res[p_ic],
                d_ic,
                weight,
                std_err,
                d_std_err,
                res["warning"],
                res[scale_col],
            )

    return df_comp.sort_values(by=ic, ascending=ascending)

In [413]:
%timeit compare_new(compare_dict,'waic')

146 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [414]:
%timeit az.stats.compare(compare_dict,'waic')

46.9 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# It's 3 times slower. I don't think that much could be done to improve bb-pseudo-bma or pseduo-bma for that matter.

In [420]:
%timeit compare_new(compare_dict,'waic',method='pseudo-bma')

122 ms ± 4.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [422]:
%timeit az.stats.compare(compare_dict,'waic',method='pseudo-bma')

17.6 ms ± 936 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [423]:
#7 times slower. :(

In [ ]:
# I guess the best method is to use _waic_new and loo_new which provide a better speedup. jitting the different methods 
# under stacking slows down the overall method substantially.

In [ ]:
"""""""""""""""""""""""""""""""""""""""""Summary"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

In [ ]:
#Summary mainly involves ufuncs which is a  part of diagnostics. I'll pick that up when I take up diagnostics
#in a couple of days. Open for suggestions and improvements.